In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, count, min, approx_count_distinct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

spark = (SparkSession.builder
         .appName("perform-aagregations")
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

In [24]:
df = (spark.read.format("csv")
      .option("header", "true")
      .option("nullValue", "null")
      .option("dateFormat", "LLLL d, y")
      .load("../data/netflix_titles.csv"))

In [25]:
# Group the data by a column
grouped_df = df.groupBy("country")

In [26]:
# Count the number of rows in each group
count_df = grouped_df.count()
count_df.show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|Peru, United Stat...|    1|
|India, United Kin...|    1|
|Japan, Canada, Un...|    1|
|United Kingdom, C...|    1|
|      India, Germany|    2|
|South Africa, Uni...|    1|
|              Russia|   15|
|United Kingdom, G...|    1|
|Chile, United Sta...|    1|
|South Africa, Angola|    1|
|United States, Po...|    1|
|  Philippines, Qatar|    1|
|Hong Kong, China,...|    1|
|  Germany, Sri Lanka|    1|
|Denmark, France, ...|    1|
|United States, Fr...|    1|
|United Kingdom, N...|    2|
|Australia, United...|    2|
|Brazil, France, G...|    1|
|United States, Ir...|    3|
+--------------------+-----+
only showing top 20 rows



In [27]:
# Apply custom aggregation using max
max_release_df = grouped_df.agg(max(col("date_added")))
max_release_df.show()

+--------------------+--------------------+
|             country|     max(date_added)|
+--------------------+--------------------+
|                null|   September 9, 2021|
|     Ama K. Abebrese|  Kobina Amissah Sam|
|         Aziz Ansari|         Carla Gallo|
|            Chuck D.|     Desiree Densiti|
|       Dominic Costa|        Nick Ferraro|
|          Doug Plaut|    Cheyenne Jackson|
|     Francesc Orella|        Imanol Arias|
|  Henri-Noël Tabary"|              France|
|       James Toback"|       United States|
| Justin ""Alyssa ...|         Molly Ryman|
|  Lachion Buckingham|       Chasity Moore|
|  Leonardo Sbaraglia|     Francesc Orella|
|   Michael Cavalieri|     Walton Goggins"|
| Remilekun ""Remi...| Charles  ""Charl...|
|          Rob Morgan|     Shakira Barrera|
|       Sophia Loren"|       United States|
|     Tantoo Cardinal|   Robert Pastorelli|
|      Theo Campbell"|       United States|
| Tobechukwu ""iLL...|          Toni Tones|
| plus Whitney Cum...|          

In [28]:
release_date_gouped_df = (
    df.groupBy("country")
    .agg(
        count("show_id").alias("NumberOfReleases")
        ,max("date_added").alias("LastReleaseDate")
        ,min("date_added").alias("FirstReleaseDate")))

release_date_gouped_df.show(3)

+----------------+----------------+-------------------+-------------------+
|         country|NumberOfReleases|    LastReleaseDate|   FirstReleaseDate|
+----------------+----------------+-------------------+-------------------+
|            null|             830|  September 9, 2021|  December 14, 2018|
| Ama K. Abebrese|               1| Kobina Amissah Sam| Kobina Amissah Sam|
|     Aziz Ansari|               1|        Carla Gallo|        Carla Gallo|
+----------------+----------------+-------------------+-------------------+
only showing top 3 rows



### Pivot Tables

In [29]:
pivot_table = (df.groupBy("country").pivot("type").agg(count("show_id")))
pivot_table.show()

+--------------------+-----+-------+
|             country|Movie|TV Show|
+--------------------+-----+-------+
|Peru, United Stat...|    1|   null|
|United Kingdom, C...| null|      1|
|India, United Kin...|    1|   null|
|      India, Germany|    2|   null|
|Japan, Canada, Un...| null|      1|
|South Africa, Uni...|    1|   null|
|              Russia|    1|     14|
|United Kingdom, G...| null|      1|
|Chile, United Sta...|    1|   null|
|  Philippines, Qatar|    1|   null|
|United States, Fr...| null|      1|
|Hong Kong, China,...|    1|   null|
|Denmark, France, ...|    1|   null|
|South Africa, Angola|    1|   null|
|United States, Po...| null|      1|
|  Germany, Sri Lanka|    1|   null|
|United Kingdom, N...|    2|   null|
|Australia, United...|    2|   null|
|United States, Ir...|    2|      1|
|Brazil, France, G...|    1|   null|
+--------------------+-----+-------+
only showing top 20 rows



### Approximate Aggregations

In [30]:
# Define a Schema
schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("ProductId", StringType(), True),
    StructField("UserId", StringType(), True),
    StructField("ProfileName", StringType(), True),
    StructField("HelpfulnessNumerator", StringType(), True),
    StructField("HelpfulnessDenominator", StringType(), True),
    StructField("Score", IntegerType(), True),
    StructField("Time", StringType(), True),
    StructField("Summary", StringType(), True),
    StructField("Text", StringType(), True)])


review_df = (spark.read.format("csv")
      .option("header",True)
      .schema(schema)
      .load("../data/Reviews.csv"))

In [31]:
# Approximate quantile calculation
quantiles = review_df.approxQuantile("Score", [0.25, 0.5, 0.75], 0.1)
print("Approximate Quantiles:", quantiles)

Approximate Quantiles: [4.0, 5.0, 5.0]


In [32]:
# Approximate distinct count calculation
approx_distinct_count = review_df.select(approx_count_distinct("ProductId", rsd=0.1).alias("approx_distinct_count"))
approx_distinct_count.show()

+---------------------+
|approx_distinct_count|
+---------------------+
|                 3780|
+---------------------+



In [33]:
spark.stop()